In [1]:
import tensorflow as tf
import numpy as np
import math

Hint: Use dtype=tf.float64 if you want to have same precision as numpy for testing<br>
Hint: You migth wanna use tf.InterativeSession for convenience

In [2]:
sess = tf.InteractiveSession()

In [3]:
def test(task_np, task_tf, *, args=()):
    expected = task_np(*args)
    phs, out_tf = task_tf()
    feed_dict = {ph: value for ph, value in zip(phs, args)}
    actual = sess.run(out_tf, feed_dict=feed_dict)
    message = f"expected: {expected}, actual: {actual}"
    isclose = np.isclose(expected, actual)
    assert isclose if isinstance(isclose, bool) else isclose.all(), message

1a: Create two random 0-d tensors x and y of any distribution. <br>
Create a TensorFlow object that returns x + y if x > y, and x - y otherwise. <br>
Hint: look up tf.cond() <br>
I do the first problem for you <br>

In [4]:
def task_1a_np(x, y):
    return np.where(x > y, x + y, x - y)

In [5]:
def task_1a_tf():
    x = tf.placeholder(tf.float64, name="x")
    y = tf.placeholder(tf.float64, name="y")
    out = tf.cond(tf.greater(x, y), lambda: tf.add(x, y), lambda: tf.subtract(x, y))
    return (x, y), out

In [6]:
test(task_1a_np, task_1a_tf, args=[np.random.uniform(50), np.random.uniform(50)])

1b: Create two 0-d tensors x and y randomly selected from the range [-1, 1).<br>
Return x + y if x < y, x - y if x > y, 0 otherwise.<br>
Hint: Look up tf.case().<br>

In [7]:
def task_1b_np(x, y):
    return np.select(condlist=[x < y, x > y],
                     choicelist=[x + y, x - y],
                     default=0)

In [8]:
def task_1b_tf():
    x = tf.placeholder(tf.float64, name="x")
    y = tf.placeholder(tf.float64, name="y")
    out = tf.case([
        (tf.less(x, y), lambda: tf.add(x, y)),
        (tf.greater(x, y), lambda: tf.subtract(x, y))
    ], default=lambda: tf.constant(0, dtype=tf.float64))
    return (x, y), out

In [9]:
test(task_1b_np, task_1b_tf, args=[np.random.uniform(50), np.random.uniform(50)])
test(task_1b_np, task_1b_tf, args=[50, 50])

1c: Create the tensor x of the value [[0, -2, -1], [0, 1, 2]] <br>
and y as a tensor of zeros with the same shape as x. <br>
Return a boolean tensor that yields Trues if x equals y element-wise. <br>
Hint: Look up tf.equal(). <br>

In [10]:
def task_1c_np():
    x = np.array([[0, -2, -1], [0, 1, 2]])
    y = np.zeros_like(x)
    return x == y

In [11]:
def task_1c_tf():
    x = tf.constant([[0, -2, -1], [0, 1, 2]], dtype=tf.float64, name="x")
    y = tf.zeros_like(x, name="y")
    out = tf.equal(x, y)
    return (), out

In [12]:
test(task_1c_np, task_1c_tf)

1d:<br>
Get the indices of elements in x whose values are greater than 30.<br>
Hint: Use tf.where().<br>
Then extract elements whose values are greater than 30.<br>
Hint: Use tf.gather().<br>

In [13]:
def task_1d_np(x):
    return x[x > 30].reshape(-1, 1)

In [14]:
def task_1d_tf():
    x = tf.placeholder(tf.float64, shape=[None], name="x")
    return (x,), tf.gather(x, tf.where(x > 30))

In [15]:
test(task_1d_np, task_1d_tf, args=[np.array([31, 0, 32])])

1e: Create a diagnoal 2-d tensor of size 6 x 6 with the diagonal values of 1,<br>
2, ..., 6<br>
Hint: Use tf.range() and tf.diag().<br>

In [16]:
def task_1e_np():
    return np.diag(np.arange(1, 7))

In [17]:
def task_1e_tf():
    return (), tf.diag(tf.range(1, 7))

In [18]:
test(task_1e_np, task_1e_tf)

1f: Create a random 2-d tensor of size 10 x 10 from any distribution.<br>
Calculate its determinant.<br>
Hint: Look at tf.matrix_determinant().<br>

In [19]:
def task_1f_np(x):
    return np.linalg.det(x)

In [20]:
def task_1f_tf():
    x = tf.placeholder(tf.float64, shape=[6, 6], name="x")
    return (x,), tf.matrix_determinant(x)

In [21]:
test(task_1f_np, task_1f_tf, args=[np.random.normal(size=[6, 6])])

1g: Create tensor x with value [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9].<br>
Return the unique elements in x<br>
Hint: use tf.unique(). Keep in mind that tf.unique() returns a tuple.<br>

In [22]:
def task_1g_np():
    x = [5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9]
    _, idx = np.unique(x, return_index=True)
    return np.take(x, sorted(idx))

In [23]:
def task_1g_tf():
    x = tf.constant([5, 2, 3, 5, 10, 6, 2, 3, 4, 2, 1, 1, 0, 9])
    return (), tf.unique(x)[0]

In [24]:
test(task_1g_np, task_1g_tf)

1h: Create two tensors x and y of shape 300 from any normal distribution,<br>
as long as they are from the same distribution.<br>
Use tf.cond() to return:<br>
- The mean squared error of (x - y) if the average of all elements in (x - y)<br>
  is negative, or<br>
- The sum of absolute value of all elements in the tensor (x - y) otherwise.<br>
Hint: see the Huber loss function in the lecture slides 3.<br>

In [25]:
def task_1h_np(x, y):
    average = np.mean(x - y)
    mse = np.mean((x - y) ** 2)
    asum = np.sum(np.abs(x - y))
    return mse if average < 0 else asum

In [26]:
def task_1h_tf():
    x = tf.placeholder(tf.float64, shape=[300], name="x")
    y = tf.placeholder(tf.float64, shape=[300], name="y")
    average = tf.reduce_mean(x - y)
    out = tf.cond(average < 0, 
                  lambda: tf.reduce_mean((x - y) ** 2), 
                  lambda: tf.reduce_sum(tf.abs(x - y)))
    return (x, y), out

In [27]:
test(task_1h_np, task_1h_tf, args=[np.random.normal(size=300), np.random.normal(size=300)])

Finally, closing the session:

In [28]:
sess.close()